In [10]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.stats import mannwhitneyu
from scipy.stats import friedmanchisquare
from scipy.stats import rankdata
import pandas as pd
import seaborn as sns

Loads Data: Iterates through a predefined list of representations (fixed-point, gray, ieee754, real-value) and attempts to load gnbg_results_<representation>.pkl files. It handles FileNotFoundError and general exceptions during loading.

Combines and Filters Results: Merges all successfully loaded results into a single list (all_combined_results). It then filters these results to include only valid runs that contain instance, representation, and a finite final_error.

Organizes Data for Analysis: Groups the valid final_error values by (instance, representation) pairs.

Performs Statistical Comparison: Conducts Mann-Whitney U tests (a non-parametric statistical test) to compare the final_error distributions between different representations for each benchmark instance. It checks for sufficient data points and handles cases with constant data.

Prints Statistical Summary: Outputs the p-values from the Mann-Whitney U tests, indicating statistical significance (at alpha=0.05), and provides an interpretation based on the medians of the compared representations.

In [11]:
print("Spojuji výsledky z jednotlivých běhů...")

representations_to_run = ['fixed-point', 'gray', 'ieee754', 'real-value']
all_combined_results = []
loaded_files_count = 0

for rep in representations_to_run:
    filename = f"gnbg_results_{rep}.pkl"
    try:
        with open(filename, 'rb') as f:
            results_for_rep = pickle.load(f)
            all_combined_results.extend(results_for_rep)
            print(f"Načteno {len(results_for_rep)} výsledků z {filename}")
            loaded_files_count += 1
    except FileNotFoundError:
        print(f"VAROVÁNÍ: Soubor {filename} nebyl nalezen. Přeskakuji.")
    except Exception as e:
        print(f"CHYBA při načítání souboru {filename}: {e}")

if not all_combined_results:
    print("CHYBA: Nepodařilo se načíst žádné výsledky. Analýza nemůže pokračovat.")
else:
    print(f"\nCelkem spojeno {len(all_combined_results)} výsledků z {loaded_files_count} souborů.")

results_by_instance_rep = {}
instances_with_results = set()
valid_results_count = 0

if all_combined_results:
    print("\nZpracovávám spojené výsledky pro analýzu...")
    for res in all_combined_results:
        if 'error' not in res and np.isfinite(res.get('final_error', float('inf'))):
            instance = res['instance']
            representation = res['representation']
            key = (instance, representation)
            if key not in results_by_instance_rep:
                results_by_instance_rep[key] = []
            results_by_instance_rep[key].append(res['final_error'])
            instances_with_results.add(instance)
            valid_results_count += 1
    instances_with_results = sorted(list(instances_with_results))
    print(f"Počet validních výsledků pro analýzu: {valid_results_count} z {len(all_combined_results)} celkem.")
    print(f"Nalezeny výsledky pro instance: {instances_with_results}")


if all_combined_results and results_by_instance_rep:
    print("\nProvádím statistické porovnání (Mann-Whitney U)...")
    statistical_comparison = {}
    alpha = 0.05

    for instance_name in instances_with_results:
        statistical_comparison[instance_name] = {}
        instance_reps = sorted(list(set(key[1] for key in results_by_instance_rep.keys() if key[0] == instance_name)))

        for i in range(len(instance_reps)):
            for j in range(i + 1, len(instance_reps)):
                rep1 = instance_reps[i]
                rep2 = instance_reps[j]
                key1 = (instance_name, rep1)
                key2 = (instance_name, rep2)
                data1 = results_by_instance_rep.get(key1, [])
                data2 = results_by_instance_rep.get(key2, [])
                min_samples_for_test = 5

                if len(data1) < min_samples_for_test or len(data2) < min_samples_for_test:
                    print(f"  INFO: Přeskakuji porovnání {rep1} vs {rep2} pro {instance_name} (málo dat: {len(data1)} vs {len(data2)})")
                    continue

                if len(set(data1)) <= 1 and len(set(data2)) <= 1:
                     u_stat, p_value = np.nan, 1.0 if np.isclose(np.mean(data1), np.mean(data2)) else 0.0
                else:
                    try:
                        u_stat, p_value = mannwhitneyu(data1, data2, alternative='two-sided', nan_policy='omit')
                    except ValueError as e:
                        print(f"  WARN: Chyba Mann-Whitney U pro {instance_name} ({rep1} vs {rep2}): {e}")
                        u_stat, p_value = np.nan, np.nan

                comparison_key = f"{rep1}_vs_{rep2}"
                statistical_comparison[instance_name][comparison_key] = {
                    'u_statistic': u_stat, 'p_value': p_value, 'n1': len(data1), 'n2': len(data2)
                }

    print("\nVýsledky statistického porovnání (p-hodnoty):")
    for instance_name in instances_with_results:
        print(f"\nInstance: {instance_name}")
        comparisons = statistical_comparison.get(instance_name, {})
        if not comparisons:
             print("  Pro tuto instanci nebylo provedeno žádné platné statistické porovnání (zkontrolujte počet běhů a validitu výsledků).")
             continue
        sorted_comparisons = sorted(comparisons.items())
        for comparison, stats in sorted_comparisons:
            p_value = stats['p_value']
            n1, n2 = stats['n1'], stats['n2']
            if np.isnan(p_value): significance_msg = "Test neprůkazný (NaN)"
            else: significant = "Ano" if p_value < alpha else "Ne"; significance_msg = f"p={p_value:.3f} (Významný: {significant})"
            print(f"  {comparison} (n={n1} vs {n2}): {significance_msg}")
            if not np.isnan(p_value) and p_value < alpha:
                 rep1, rep2 = comparison.split('_vs_')
                 data1 = results_by_instance_rep.get((instance_name, rep1), []); data2 = results_by_instance_rep.get((instance_name, rep2), [])
                 median1 = np.median(data1) if data1 else np.nan; median2 = np.median(data2) if data2 else np.nan
                 if not np.isnan(median1) and not np.isnan(median2):
                      if np.isclose(median1, median2): interpretation = "Mediány blízké."
                      elif median1 < median2: interpretation = f"'{rep1}' lepší (nižší medián)."
                      else: interpretation = f"'{rep2}' lepší (nižší medián)."
                      print(f"      Mediány: {median1:.3e} vs {median2:.3e} -> {interpretation}")

Spojuji výsledky z jednotlivých běhů...
Načteno 744 výsledků z gnbg_results_fixed-point.pkl
Načteno 744 výsledků z gnbg_results_gray.pkl
Načteno 744 výsledků z gnbg_results_ieee754.pkl
Načteno 744 výsledků z gnbg_results_real-value.pkl

Celkem spojeno 2976 výsledků z 4 souborů.

Zpracovávám spojené výsledky pro analýzu...
Počet validních výsledků pro analýzu: 2976 z 2976 celkem.
Nalezeny výsledky pro instance: ['f1', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f2', 'f20', 'f21', 'f22', 'f23', 'f24', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']

Provádím statistické porovnání (Mann-Whitney U)...

Výsledky statistického porovnání (p-hodnoty):

Instance: f1
  fixed-point_vs_gray (n=31 vs 31): p=0.000 (Významný: Ano)
      Mediány: 2.041e+03 vs 5.672e+01 -> 'gray' lepší (nižší medián).
  fixed-point_vs_ieee754 (n=31 vs 31): p=0.001 (Významný: Ano)
      Mediány: 2.041e+03 vs 3.312e+03 -> 'fixed-point' lepší (nižší medián).
  fixed-point_vs_real-value (n=31 vs 31): p

Data Consolidation: Loads and combines results, similar to the previous cell, but focuses on best_fitness_found.

Friedman Rank Test: For each unique benchmark instance, it conducts a Friedman test. This non-parametric test determines if there's a statistically significant difference among the multiple genetic algorithm representations based on their ranks (performance) across the experimental runs.

Data Preprocessing: Before the Friedman test, it checks if enough data is available for each representation and, if necessary, truncates the number of runs to ensure all compared representations have an equal number of samples, which is a requirement for the Friedman test.

Post-Hoc Nemenyi Test: If the Friedman test indicates a statistically significant difference, it then performs a Nemenyi post-hoc test. This test is used for pairwise comparisons between the representations to identify which specific pairs of representations show a significant difference. It calculates a Critical Difference (CD) value, and if the absolute difference in average ranks between two representations exceeds this CD, their performance is considered statistically different.

Output: Prints detailed results of both the Friedman and Nemenyi tests, including chi-squared statistics, p-values, degrees of freedom, critical difference values, average ranks, and clear statements about statistical significance and performance interpretation.

In [12]:
print("Spojuji výsledky z jednotlivých běhů...")

representations_to_run = ['fixed-point', 'gray', 'ieee754', 'real-value']

all_combined_results = []
loaded_files_count = 0

for rep in representations_to_run:
    filename = f"gnbg_results_{rep}.pkl"
    try:
        with open(filename, 'rb') as f:
            results_for_rep = pickle.load(f)
            all_combined_results.extend(results_for_rep)
            print(f"Načteno {len(results_for_rep)} výsledků z {filename}")
            loaded_files_count += 1
    except FileNotFoundError:
        print(f"VAROVÁNÍ: Soubor {filename} nebyl nalezen. Přeskakuji.")
    except Exception as e:
        print(f"CHYBA při načítání souboru {filename}: {e}")

if not all_combined_results:
    print("CHYBA: Nepodařilo se načíst žádné výsledky. Analýza nemůže pokračovat.")
else:
    print(f"\nCelkem spojeno {len(all_combined_results)} výsledků z {loaded_files_count} souborů.")

    results_by_instance_rep = {}
    instances_with_results = set()

    for run in all_combined_results:
        if 'instance' in run and 'representation' in run and 'best_fitness_found' in run:
            instance_name = run['instance']
            representation_name = run['representation']
            best_fitness = run['best_fitness_found']

            key = (instance_name, representation_name)
            if key not in results_by_instance_rep:
                results_by_instance_rep[key] = []
            results_by_instance_rep[key].append(best_fitness)
            instances_with_results.add(instance_name)
        else:
            pass

    instances_with_results = sorted(list(instances_with_results))
    all_representations_found = sorted(list(set(key[1] for key in results_by_instance_rep.keys())))
    
    print(f"\nData připravena pro Friedmanův test.")
    print(f"Nalezené unikátní instance (problémové funkce): {instances_with_results}")
    print(f"Nalezené unikátní reprezentace (metody): {all_representations_found}")


    print("\n--- Friedman Rank Test (porovnání reprezentací pro každou instanci) ---")
    alpha = 0.05

    # Pro k = 2, 3, 4, 5, 6, 7, 8, 9, 10
    # Values from: https://www.stat.auckland.ac.nz/~wild/ChanceEnc/Ch10.friedman-test.pdf (or similar)
    # Critical values q_alpha for alpha=0.05 (two-tailed) and various k values (number of groups)
    # k:    2       3       4       5       6       7       8       9       10
    q_critical_values_nemenyi = {
        2: 1.960, 3: 2.343, 4: 2.569, 5: 2.728, 6: 2.850, 7: 2.950, 8: 3.030, 9: 3.099, 10: 3.161
    }


    if len(all_representations_found) < 2:
        print(f"INFO: Pro Friedmanův test je potřeba alespoň 2 reprezentace k porovnání. Nalezeno: {len(all_representations_found)}.")
    elif len(instances_with_results) < 1:
        print("INFO: Pro Friedmanův test je potřeba alespoň 1 instance. Žádné instance nalezeny.")
    else:
        for instance_name in instances_with_results:
            print(f"\n----- Instance: {instance_name} -----")
            
            data_for_friedman_instance = []
            reps_included_in_test = []

            for rep_name in all_representations_found:
                key = (instance_name, rep_name)
                data_points = results_by_instance_rep.get(key, [])
                
                if data_points:
                    data_for_friedman_instance.append(data_points)
                    reps_included_in_test.append(rep_name)
                else:
                    print(f"  INFO: Reprezentace '{rep_name}' nemá data pro instanci '{instance_name}' a bude přeskočena v tomto testu.")

            if len(data_for_friedman_instance) < 2:
                print(f"  INFO: Pro instanci '{instance_name}' není dostatek reprezentací s platnými daty k porovnání ({len(reps_included_in_test)} nalezeno). Test přeskočen.")
                continue

            num_runs_per_rep = [len(d) for d in data_for_friedman_instance]
            if len(set(num_runs_per_rep)) > 1:
                print(f"  VAROVÁNÍ: Pro instanci '{instance_name}' se počty běhů pro porovnávané reprezentace liší ({num_runs_per_rep}).")
                min_runs = min(num_runs_per_rep)
                data_for_friedman_instance = [d[:min_runs] for d in data_for_friedman_instance]
                print(f"  Data byla oříznuta na {min_runs} běhů pro každou reprezentaci pro instanci '{instance_name}'.")
            else:
                min_runs = num_runs_per_rep[0]

            if min_runs < 2:
                print(f"  INFO: Pro instanci '{instance_name}' je po oříznutí méně než 2 běhy pro každou reprezentaci. Test přeskočen.")
                continue

            try:
                chi_squared_stat, p_value = friedmanchisquare(*data_for_friedman_instance)

                print(f"  Chi-squared statistika: {chi_squared_stat:.4f}")
                print(f"  P-hodnota: {p_value:.4f}")
                print(f"  Stupně volnosti (df): {len(reps_included_in_test) - 1}")

                if p_value < alpha:
                    print(f"  Výsledek je statisticky významný (p < {alpha}).")
                    print(f"  => Existuje statisticky významný rozdíl mezi reprezentacemi pro instanci '{instance_name}'.")
                    
                    print("\n  --- Post-hoc Nemenyiho test (porovnání párů reprezentací) ---")
                    
                    k = len(reps_included_in_test)
                    N = min_runs
                    
                    if k < 2 or N < 1:
                        print("  INFO: Nedostatek dat pro provedení Nemenyiho testu.")
                    else:
                        q_alpha = q_critical_values_nemenyi.get(k)
                        if q_alpha is None:
                            print(f"  VAROVÁNÍ: Kritická hodnota pro k={k} není k dispozici. Nemenyiho test nelze provést.")
                        else:
                            CD = q_alpha * np.sqrt(k * (k + 1) / (6 * N))
                            print(f"  Kritická hodnota rozdílu (CD) pro Nemenyiho test (alpha={alpha}, k={k}, N={N}): {CD:.4f}")

                            transposed_runs_data = list(zip(*data_for_friedman_instance))

                            ranks_per_run = [rankdata(row) for row in transposed_runs_data]

                            mean_ranks = np.mean(ranks_per_run, axis=0)
                            
                            print("\n  Průměrné ranky reprezentací:")
                            for i, rep_name in enumerate(reps_included_in_test):
                                print(f"    {rep_name}: {mean_ranks[i]:.4f}")

                            print("\n  Výsledky párového porovnání (rozdíl průměrných ranků vs. CD):")
                            for i in range(len(reps_included_in_test)):
                                for j in range(i + 1, len(reps_included_in_test)):
                                    rep1_name = reps_included_in_test[i]
                                    rep2_name = reps_included_in_test[j]
                                    
                                    diff = abs(mean_ranks[i] - mean_ranks[j])
                                    
                                    significance_msg = ""
                                    if diff > CD:
                                        significance_msg = f"Statisticky významný rozdíl (> CD)"
                                    else:
                                        significance_msg = f"Není významný rozdíl (<= CD)"
                                    
                                    print(f"    {rep1_name} vs {rep2_name}: Rozdíl={diff:.4f} ({significance_msg})")

                else:
                    print(f"  Výsledek není statisticky významný (p >= {alpha}).")
                    print(f"  => Neexistuje statisticky významný rozdíl mezi reprezentacemi pro instanci '{instance_name}'.")
                    print("  Post-hoc test není potřeba, protože celkový test neprokázal významný rozdíl.")

            except ValueError as e:
                print(f"  WARN: Chyba při provádění Friedmanova testu pro instanci '{instance_name}': {e}")
                print("  Zkontrolujte, zda mají všechny skupiny (reprezentace) dostatek dat a jsou platné.")
            except Exception as e:
                print(f"  Neočekávaná chyba při Friedmanově testu pro instanci '{instance_name}': {e}")

Spojuji výsledky z jednotlivých běhů...
Načteno 744 výsledků z gnbg_results_fixed-point.pkl
Načteno 744 výsledků z gnbg_results_gray.pkl
Načteno 744 výsledků z gnbg_results_ieee754.pkl
Načteno 744 výsledků z gnbg_results_real-value.pkl

Celkem spojeno 2976 výsledků z 4 souborů.

Data připravena pro Friedmanův test.
Nalezené unikátní instance (problémové funkce): ['f1', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f2', 'f20', 'f21', 'f22', 'f23', 'f24', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']
Nalezené unikátní reprezentace (metody): ['fixed-point', 'gray', 'ieee754', 'real-value']

--- Friedman Rank Test (porovnání reprezentací pro každou instanci) ---

----- Instance: f1 -----
  Chi-squared statistika: 83.3226
  P-hodnota: 0.0000
  Stupně volnosti (df): 3
  Výsledek je statisticky významný (p < 0.05).
  => Existuje statisticky významný rozdíl mezi reprezentacemi pro instanci 'f1'.

  --- Post-hoc Nemenyiho test (porovnání párů reprezentací) ---
  Kritická ho

It attempts to load .pkl files (e.g., gnbg_results_fixed-point.pkl) for a predefined set of genetic algorithm representations (fixed-point, gray, ieee754, real-value).
It includes robust error handling for FileNotFoundError and other general exceptions during the loading process, printing informative warnings or errors.
All successfully loaded results from different representations are combined into a single dataset.
Data Filtering and Preparation:

It filters the combined results to ensure that only entries containing essential information (instance, representation, and final_error) are included for analysis.
These filtered results are then converted into a pandas.DataFrame.
The final_error column is explicitly converted to a numeric type, and any rows with non-numeric or missing final_error values are removed.
It performs checks to ensure that valid data remains after filtering and cleaning.
Statistical Summary of Final Error:

It groups the data by instance (benchmark problem) and representation.
For each group, it calculates and displays key descriptive statistics for final_error: minimum, maximum, mean (average), standard deviation, and count of runs.
These statistical values are formatted into scientific notation for clarity.
Graphical Visualization (Box Plots):

It sets up seaborn for plotting with a whitegrid style.
It creates a plots directory if it doesn't already exist to store the generated graphs.
For each unique benchmark instance, it generates a box plot:
The x-axis represents the different genetic algorithm representations.
The y-axis represents the final_error, which is displayed on a logarithmic scale (log) to accommodate wide ranges of error values.
Each box plot visualizes the distribution (median, quartiles, and outliers) of the final_error for each representation.
Mean final_error values are overlaid as red triangular scatter points on the box plots for easy comparison.
Each plot is given a descriptive title and axis labels.
Plots are saved as PNG image files (e.g., final_error_InstanceName.png) within the plots directory.

In [13]:

print("Spojuji výsledky z jednotlivých běhů...")

representations_to_run = ['fixed-point', 'gray', 'ieee754', 'real-value']

all_combined_results = []
loaded_files_count = 0

for rep in representations_to_run:
    filename = f"gnbg_results_{rep}.pkl"
    try:
        with open(filename, 'rb') as f:
            results_for_rep = pickle.load(f)
            all_combined_results.extend(results_for_rep)
            print(f"Načteno {len(results_for_rep)} výsledků z {filename}")
            loaded_files_count += 1
    except FileNotFoundError:
        print(f"VAROVÁNÍ: Soubor {filename} nebyl nalezen. Přeskakuji.")
    except Exception as e:
        print(f"CHYBA při načítání souboru {filename}: {e}")

if not all_combined_results:
    print("CHYBA: Nepodařilo se načíst žádné výsledky. Nelze pokračovat s analýzou final_error.")
else:
    print(f"\nCelkem spojeno {len(all_combined_results)} výsledků z {loaded_files_count} souborů.")

    filtered_results = [
        run for run in all_combined_results
        if 'instance' in run and 'representation' in run and 'final_error' in run
    ]

    if not filtered_results:
        print("CHYBA: V načtených datech nebyly nalezeny žádné záznamy s 'final_error'.")
    else:
        df_final_error = pd.DataFrame(filtered_results)

        df_final_error['final_error'] = pd.to_numeric(df_final_error['final_error'], errors='coerce')
        df_final_error.dropna(subset=['final_error'], inplace=True)

        if df_final_error.empty:
            print("CHYBA: Po filtrování a čištění dat pro 'final_error' nezůstala žádná platná data.")
        else:
            print(f"\nDataFrame pro 'final_error' vytvořen s {len(df_final_error)} záznamy.")
            print(f"Unikátní instance pro analýzu: {df_final_error['instance'].unique()}")
            print(f"Unikátní reprezentace pro analýzu: {df_final_error['representation'].unique()}")

            print("\n--- Statistiky Final Error (Min, Max, Průměr, Směrodatná odchylka) ---")

            stats_final_error = df_final_error.groupby(['instance', 'representation'])['final_error'].agg(
                min_error='min',
                max_error='max',
                mean_error='mean',
                std_error='std',
                count='count'
            ).reset_index()

            stats_final_error['mean_error'] = stats_final_error['mean_error'].apply(lambda x: f"{x:.4e}")
            stats_final_error['std_error'] = stats_final_error['std_error'].apply(lambda x: f"{x:.4e}")
            stats_final_error['min_error'] = stats_final_error['min_error'].apply(lambda x: f"{x:.4e}")
            stats_final_error['max_error'] = stats_final_error['max_error'].apply(lambda x: f"{x:.4e}")

            print(stats_final_error.to_string(index=False))


            print("\n--- Grafické zobrazení Final Error ---")

            sns.set_theme(style="whitegrid")

            if not os.path.exists('plots'):
                os.makedirs('plots')

            for instance_name in df_final_error['instance'].unique():
                fig, ax = plt.subplots(figsize=(10, 6))

                subset_df = df_final_error[df_final_error['instance'] == instance_name]

                sns.boxplot(x='representation', y='final_error', data=subset_df, ax=ax, hue='representation', palette='viridis', legend=False)

                ax.set_title(f'Final Error Distribution for {instance_name}')
                ax.set_xlabel('Representation')
                ax.set_ylabel('Final Error')
                ax.set_yscale('log')

                mean_values = subset_df.groupby('representation')['final_error'].mean().reset_index()
                sns.scatterplot(x='representation', y='final_error', data=mean_values, ax=ax,
                                marker='^', color='red', s=100, zorder=5, label='Mean')

                ax.legend(title='Metrics')

                plt.tight_layout()

                safe_instance_name = instance_name.replace(" ", "_").replace("/", "_").replace("\\", "_")
                plot_filename = f"plots/final_error_{safe_instance_name}.png"
                plt.savefig(plot_filename)
                print(f"Graf pro instanci '{instance_name}' uložen jako: {plot_filename}")

                plt.close(fig)

            print("\nVšechny grafy byly úspěšně uloženy do složky 'plots'.")

Spojuji výsledky z jednotlivých běhů...
Načteno 744 výsledků z gnbg_results_fixed-point.pkl
Načteno 744 výsledků z gnbg_results_gray.pkl
Načteno 744 výsledků z gnbg_results_ieee754.pkl
Načteno 744 výsledků z gnbg_results_real-value.pkl

Celkem spojeno 2976 výsledků z 4 souborů.

DataFrame pro 'final_error' vytvořen s 2976 záznamy.
Unikátní instance pro analýzu: ['f1' 'f10' 'f11' 'f12' 'f13' 'f14' 'f15' 'f16' 'f17' 'f18' 'f19' 'f2'
 'f20' 'f21' 'f22' 'f23' 'f24' 'f3' 'f4' 'f5' 'f6' 'f7' 'f8' 'f9']
Unikátní reprezentace pro analýzu: ['fixed-point' 'gray' 'ieee754' 'real-value']

--- Statistiky Final Error (Min, Max, Průměr, Směrodatná odchylka) ---
instance representation  min_error  max_error mean_error  std_error  count
      f1    fixed-point 6.2990e+02 4.4375e+03 2.0655e+03 9.2617e+02     31
      f1           gray 6.0790e-01 9.9574e+02 1.2300e+02 2.0103e+02     31
      f1        ieee754 2.3378e+02 8.8610e+03 3.6241e+03 2.0776e+03     31
      f1     real-value 1.1812e-04 4.9758e-04